In [6]:
import sympy as sp
import numpy as np
import scipy.optimize as opt
from copy import copy
import random as rd
from matplotlib import pyplot as plt
sp.init_printing(use_latex="mathjax")

# Aplicar ponto numa função vetorial e retornar o vetor
def point_in_func (func: list, symbols: list, point: list):
    func_in_point = []
    for i in range(0, len(func)):
        func_in_point.append(copy(func[i]))
        for j in range (0, len(symbols)):
            func_in_point[i] = func_in_point[i].subs(symbols[j], point[j])
            
    return func_in_point

# Retornar o produto escalar de dois vetores
def dot_product (v1: list, v2: list):
    result = 0.0
    
    if len(v1) == len (v2):
        for i in range (0, len(v1)):
            result += v1[i]*v2[i]
    
    return result

# Retornar a norma ao quadrado de um vetor
def squared_norm (Xn: list):
    return dot_product(Xn, Xn)

# Retornar o vetor v1-v2
def vec_sub (v1: list, v2: list):
    result = []
    if (len(v1) == len (v2)):
        for i in range (0, len(v1)):
            result.append(v1[i]-v2[i])
    
    return result

# Retornar novo vetor Xn
def get_new_xk (xk: list, step_size, direction: list):
    new_xk = []
    for i in range (0, len(xk)):
        new_xk.append( xk[i] + step_size*direction[i] )
    return new_xk

In [5]:
n = 5

In [3]:
# Criando as variáveis:
x = sp.symbols('x:'+ str(n))
display (x)

# Criando variável direção de busca:
d = sp.symbols('d:'+ str(n))
display (d)

(x₀, x₁, x₂, x₃, x₄)

(u₀, u₁, u₂, u₃, u₄)

(d₀, d₁, d₂, d₃, d₄)

Digite as funções fx (derivável) e gx (convexa) usando x[i] para cada coordenada i de um elemento x do domínio:

In [6]:
fx = [x[1]**2, x[0]**2, x[0]**2 + x[1]**2]
gx = [sp.Abs(x[1]-2), sp.Abs(x[0]+3), sp.Abs(x[0]-1) + sp.Abs(x[1]+1)]

In [7]:
# Gerando Fx = fx + gx

m = len (fx)

Fx = []
for i in range (0, m):
    Fx.append(fx[i] + gx[i])
print ("Fx:")
display (Fx)

Fx:


⎡  2               2               2     2                      ⎤
⎣x₁  + │x₁ - 2│, x₀  + │x₀ + 3│, x₀  + x₁  + │x₀ - 1│ + │x₁ + 1│⎦

In [8]:
# Calculando a função psix_d_i = grad(fx_i)*d + g_i(x+d) - g_i(x), para todo i

# Calculando a jacobiana de fx:
fx = sp.Matrix(fx)
print ("fx:")
display (fx)
Jfx = fx.jacobian(x)
print ("JFx (Jacobiana de fx):")
display (Jfx)

# Construindo g(x+d):
x_plus_d = []
for i in range (0,n):
    x_plus_d.append(d[i]+x[i])
gx_d = point_in_func (gx, x, x_plus_d)
print ("g(x+d):")
display(gx_d)

# Construindo pré psix(d):
pre_psix_d = []
for i in range (0,m):
    pre_psix_d.append(dot_product(Jfx.row(i),d) + gx_d[i] - gx[i])
print ("psix(d):")
display(pre_psix_d)

fx:


⎡     2   ⎤
⎢   x₁    ⎥
⎢         ⎥
⎢     2   ⎥
⎢   x₀    ⎥
⎢         ⎥
⎢  2     2⎥
⎣x₀  + x₁ ⎦

JFx (Jacobiana de fx):


⎡ 0    2⋅x₁⎤
⎢          ⎥
⎢2⋅x₀   0  ⎥
⎢          ⎥
⎣2⋅x₀  2⋅x₁⎦

g(x+d):


[│d₁ + x₁ - 2│, │d₀ + x₀ + 3│, │d₀ + x₀ - 1│ + │d₁ + x₁ + 1│]

psix(d):


[2⋅d₁⋅x₁ - │x₁ - 2│ + │d₁ + x₁ - 2│, 2⋅d₀⋅x₀ - │x₀ + 3│ + │d₀ + x₀ + 3│, 2⋅d₀⋅
x₀ + 2⋅d₁⋅x₁ - │x₀ - 1│ - │x₁ + 1│ + │d₀ + x₀ - 1│ + │d₁ + x₁ + 1│]

In [9]:
# A função objetivo phi(d) do subproblema para encontrar a direção dk:
def obj_func (d: list):
    # Retorna psix(d) aplicada em d 
    pre_psix_d_in_x = pre_psix_d_in_x_func(d)
    global l # Referenciamos l de fora da função
    return max(pre_psix_d_in_x) + (l/2)*squared_norm (d)

In [10]:
# Método Proximal-Gradiente Multiobjetivo (Rn -> Rm)
# NÃO FUNCIONANDO POR CONTA DO SUBPROBLEMA

error = 10**(-6)
l = 1 # Constante positiva para minimizar o sobproblema e encontrar a direção do passo
p = 0.5 # Constante entre 0 e 1 para realizar a Regra de Armijo
E = 0.5 # Constante entre 0 e 1 pela qual multiplicamos o tamanho do passo para diminuí-lo

q_walks = 50
all_walks = []
for i_walk in range(0, q_walks):

    new_xk = []
    for j in range (0, n):
            new_xk.append((2*rd.random()-1.0)*100) # r.random() retorna um real entre 0 e 1
    
    print ("xk =", new_xk)       
    walk = [] # Lista com todos os pontos gerados no caminhamento
    while True:
        xk = new_xk # Se xk != new_xk, xk recebe new_xk e fazemos mais um passo

        # Obtendo Fx(xk)
        Fx_in_xk = point_in_func(Fx, x, xk) # Fx aplicada em xk
        walk.append(Fx_in_xk) # Salvando o caminhamento

        # Computando a direção de minimização:

        # Gerando a função phi(d) para otimizar como subproblema e encontrar a direção d
        pre_psix_d_in_x = point_in_func(pre_psix_d, x, xk) # Aplicando xk para criar a função objetivo
        pre_psix_d_in_x_func = sp.lambdify([d], pre_psix_d_in_x,'numpy') # Criando a função lambda

        # Criando um d0 inicial aleatório para a otimização quadrática de phi(d)
        d0 = []
        for j in range (0, n):
            d0.append((2*rd.random()-1.0)) # r.random() retorna um real entre 0 e 1

        # Realizamos a minimização de phi(d) e obtemos a direção de minimizaçaõ dk
        # É ESTE SUBPROBLEMA QUE NÃO PERMITE O MÉTODO FUNCIONAR
        dk = opt.minimize(obj_func, d0, method='Powell').x

        # print ("dk =",dk)

        # Computando o tamanho do passo:

        # Obtendo psi_xk(dk)
        psi_x_in_dk = max(pre_psix_d_in_x_func(dk)) # Pegamos o máximo de psi_xk(dk)    

        tk=1 # O tamanho do passo começa com 1
        while (True):
            # Preparando para verificar a Regra de Armijo
            new_xk = get_new_xk(xk, tk, dk) # new_xk = xk + tk*dk
            Fx_in_new_xk = point_in_func(Fx, x, new_xk) # Fx aplicada em new_xk

            # print ("Fx(xk) =", Fx_in_xk, "Fx(xk+1) =", Fx_in_new_xk, "tk =", tk)

            tk_is_valid = True
            for i in range(0, m):
                # Se não respeitar a regra de armijo, para de verificar e indica que é inválido
                if Fx_in_new_xk[i] > Fx_in_xk[i] + tk*p*psi_x_in_dk: 
                    tk_is_valid = False
                    break

            # Se tk for válido para a Regra de Armijo, usamos ele
            if tk_is_valid:
                break
            else:
                tk = E*tk # Diminuímos tk se não for válido

        # Ao chegar aqui, o new_xk calculado respeita a Regra de Armijo

        # Paramos quando a diferença do x entre o passo atual e o anterior é muito pequena
        if (squared_norm(vec_sub(xk,new_xk)) <= error**2):
            break

        # Printando o new_xk encontrado e a norma quadrada da diferença com o xk 
        print ('new_xk =',new_xk,', ||xk-new_xk||² =',squared_norm(vec_sub(xk,new_xk)))

    print ("x =",new_xk)
    all_walks.append(walk)

xk = [-86.83905949310994, -90.03970303769353]
new_xk = [-41.67764917701478, -46.51058970162527] , ||xk-new_xk||² = 3934.336689562982
new_xk = [-20.044970245056753, -27.15066049875138] , ||xk-new_xk||² = 842.7796565134698
new_xk = [-19.59182504141655, -26.813964934973303] , ||xk-new_xk||² = 0.3187044782499608
new_xk = [-18.663146759165862, -26.13084032779623] , ||xk-new_xk||² = 1.3291025808549177
new_xk = [-6.514934472215195, -17.55126367153776] , ||xk-new_xk||² = 221.18819736943436
new_xk = [-1.1105121551502366, -15.653074494485686] , ||xk-new_xk||² = 32.81090273306741
new_xk = [-0.5000000043014965, -15.629999785596524] , ||xk-new_xk||² = 0.37325752852427435
x = [-0.5000000043013378, -15.629999785595276]
xk = [83.7245165962982, 72.00600904115416]
new_xk = [48.284321000809456, 30.751709197769806] , ||xk-new_xk||² = 2957.924719414362
new_xk = [34.42584015870699, 8.8785410055185] , ||xk-new_xk||² = 670.4929780174353
new_xk = [32.44499760109102, 1.0319113000943894] , ||xk-new_xk||² = 65.49

In [11]:
# Para abrir a janela interativa do matplotlib, use:
%matplotlib qt 

# Plot 2D para funções com domínio R1
if m == 2:
    plt.figure()
    plt.xlabel("F1(x)")
    plt.ylabel("F2(x)")
    
    for i_walk in range (0, q_walks):
        walk_x_axis = []
        walk_y_axis = []

        # Plotando as caminhadas
        for j in range (0,len(all_walks[i_walk])):
            walk_x_axis.append(all_walks[i_walk][j][0])
            walk_y_axis.append(all_walks[i_walk][j][1])
        plt.plot (walk_x_axis, walk_y_axis, marker = 'o', linestyle = '-')
    
    # Plotando:
    plt.show()

# Plot 3D para funções com domínio R2
elif m == 3:
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_xlabel('f1')
    ax.set_ylabel('f2')
    ax.set_zlabel('f3')
    
    for i_walk in range (0, q_walks):
        walk_x_axis = []
        walk_y_axis = []
        walk_z_axis = []

        # Plotando as caminhadas
        for j in range (0,len(all_walks[i_walk])):
            walk_x_axis.append(all_walks[i_walk][j][0])
            walk_y_axis.append(all_walks[i_walk][j][1])
            walk_z_axis.append(all_walks[i_walk][j][2])
        ax.plot (walk_x_axis, walk_y_axis, marker = 'o', linestyle = '-')
    
    # Plotando:
    plt.show()